<a href="https://colab.research.google.com/github/eluisluzquadros/covid_impact_se/blob/main/step01_1_collection_grey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalação das dependencias
#### SEMPRE RODAR ANTES

In [10]:

#!apt-get update # to update ubuntu to correctly run apt install


#'''
# scraper
!pip install selenium
!apt install chromium-chromedriver
!pip install webdriver-manager
!cp /usr/lib/chromium-browser/chromedriver/usr/bin

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium msticpy tld folium
!pip pymongo pymongo[srv] dnspython

# geocoder ipaddress
!pip install msticpy

# get domain
!pip3 install tld


# spacy
!pip install -U pip setuptools wheel
!pip install -U spacy
#!python -m spacy download en
#!python -m spacy download pt
!python -m spacy download en_core_web_sm
# spacy contextual Spell Check
!pip install contextualSpellCheck
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3

#'''


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (103.0.5060.134-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 76 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: missing destination file operand after '/usr/lib/chromium-browser/chromedriver/usr/bin'
Try 'cp --help' for more information.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://secur

### Import das dependencias

In [11]:
import os
import glob
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, date, timedelta

import urllib
import socket
from msticpy.sectools.geoip import IPStackLookup

import spacy
import contextualSpellCheck

import pymongo
import numpy as np
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta
from pandas.tseries.offsets import DateOffset
import dns

import tld
from tld import get_tld, get_fld

from google.colab import drive

### Connect google drive

In [12]:
# Codigo para montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#Caminho no google drive
path = "/content/drive/My Drive/covid19impactSE/data/01_grey/2022Sample/01_collect/en/"

# Listar os arquivos da pasta no google drive
os.chdir(path)
!ls

2021-12-31.csv


### Import chaves de api

In [14]:
# drive
keys = pd.read_csv("/content/drive/My Drive/covid19impactSE/keys/credentials.csv", sep=';')
access_token_ipstack = keys.loc[0,"key"]
access_token_pymongo = keys.loc[1,"key"]

### Create functions

In [15]:
GSE_RESULTS_CLASS = 'g Ww4FFb tF2Cxc'
GSE_MAX_RESULTS = 200

NLP = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(NLP)

def search(query, date):

  # set chrome_options for chrome driver
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

  # define url de busca
  d = date.strftime('%Y%m%d')
  d_after = (date - DateOffset(days = 1)).date()
  d_before = (date + DateOffset(days = 1)).date()
  q = "{} after:{} and before:{}".format(query, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'))
  url = "https://www.google.com/search?q={}&sort=date:r:{}:{}&num={}".format(q, d_after.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'), GSE_MAX_RESULTS)
  print(url)
  print("horizon = "+"after:"+ d_after.strftime('%Y-%m-%d') +" and "+"before:"+d_before.strftime('%Y-%m-%d'))
  #print(query)
  print('\n')

  # get url com driver
  driver.get(url)
  # passa o html para o BeautifulSoup
  soup = BeautifulSoup(driver.page_source,'html.parser') 
  # busca a tag principal das news do google earth engine
  items = soup.find_all('div',{'class':GSE_RESULTS_CLASS})

  # copia o conteudo da class 'GSE_RESULTS_CLASS' para uma lista
  result = []
  print('counts : ' + str(len(items)))

  for each in items:
    try:
      header = each.find('div',{'class':'yuRUbf'}).find('h3').text
      summary = each.find('div',{'class':'VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf'}).find('span').find('span').text,
      link = each.find('a', href=True)['href']

      
      result.append({'date' : date.strftime('%Y-%m-%d'),
                     'header' : header,
                     'summary': summary,
                     'link' : link})
    except:
      pass

  # cria um csv com a lista  
  df = pd.DataFrame(result)
  return df

def fetch_domain(df):
  df['protocol'],df['domain'],df['path'],df['query'],df['fragment'] = zip(*[urllib.parse.urlsplit(x) for x in df.link])
  return df

def fetch_hostname(df): 
  urls = df['link']
  hostnames = []
  for url in urls:
    try:
      res = get_tld(url, as_object=True)
      hostnames.append(res.fld)
    except:
      res = np.nan
      hostnames.append(res)
  df['host_name'] = hostnames
  return df

def fetch_ipaddress(df):
  domains = df['domain']
  ipaddress = []
  for i in domains:
      i_strip = i.strip()
      try:
          ip = socket.gethostbyname(i_strip)
          ipaddress.append(ip)
      except socket.gaierror:
        ipaddress.append('127.0.0.1')
  df['ip_address'] = ipaddress
  return df

def fetch_location(df):
  iplocation = IPStackLookup(api_key=access_token_ipstack)
  df = iplocation.df_lookup_ip(df, column="ip_address")
  return df

def fetch_nlp(df):
  df['header_context_true'] = df['header'].apply(lambda x: str(NLP(x)))
  df['summary_context_true'] = df['summary'].apply(lambda x: str(NLP(x)))
  return df

def save(df):
  datamax = str(df['date'].max())
  fileName = path + datamax + '.csv'
  df.to_csv(fileName, sep=';', encoding='UTF-8')
  
def next_data(df):
  df['date'] = pd.to_datetime(df['date'], dayfirst = False)
  lst_date = df['date'].max()
  nxt_date = (lst_date.date() + DateOffset(days = 1)).date()
  return nxt_date

### APENAS PARA MONGODB ###
#def save(df):
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = df.astype('str')
#  db.impacts.insert_many(df.to_dict('records'))

#def load():
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = pd.DataFrame.from_records(db.impacts.find())
# return df

### Create strings

In [16]:
# Scraper 1 (2020)
keyword = '"covid impact" OR "disaster" OR "resilience" AND "software engineers" OR "software development" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working"'

### Merge csv's or SCRAPER new search

In [18]:
# merge multiples csv's files
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
files = len(all_filenames)

if files >= 1:
  #combine all files in path
  df = pd.concat([pd.read_csv(f, sep=';', parse_dates=["date"], engine="python", encoding='UTF-8') for f in all_filenames ])

  #get next data
  next_date = next_data(df)

  #get seach
  df_new = search(keyword, next_date)

  #get domain
  df_new = fetch_domain(df_new)

  #get hostname
  df_new = fetch_hostname(df_new)

  #get ipaddress
  df_new = fetch_ipaddress(df_new)

  #get geocoded ipaddress
  df_new = fetch_location(df_new)

  #get nlp contextualSpellCheck 
  #df_new = fetch_nlp(df_new)

  #save df new
  save(df_new)

  #print new df
  df_new.head(2)
  
else:
  #get first date
  first_date = date(2021, 12, 31)
  #get seach
  df = search(keyword,first_date)
  #get domain
  df = fetch_domain(df)
  #get hostname
  df = fetch_hostname(df)
  #get ipaddress
  df = fetch_ipaddress(df)
  #get geocoded ipaddress
  df = fetch_location(df)
  #save
  save(df)  

#print df
df.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
https://www.google.com/search?q="covid impact" OR "disaster" OR "resilience" AND "software engineers" OR "software development" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working" after:2022-01-01 and before:2022-01-02&sort=date:r:2021-12-31:2022-01-02&num=200
horizon = after:2021-12-31 and before:2022-01-02


counts : 24


,Unnamed: 0,date,header,summary,link,protocol,domain,path,query,fragment,...,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,IpAddress
54,54,2021-12-31,Muthoot FinCorp Limited ANNUAL REPORT 2020 - 21,"('Dec 31, 2021',)",https://www.muthootfincorp.com/wp-content/uplo...,https,www.muthootfincorp.com,/wp-content/uploads/2021/12/Annual-Report-2020...,NaN,NaN,...,US,United States,Michigan,Southfield,-83.288857,42.466061,16509,2022-08-03 12:37:19.336950,geolocation,35.71.161.138
55,55,2021-12-31,annual report 2019 - 2020 - BIPA,"('Jan 1, 2022',)",https://www.bipa.na/?wpdmact=process&did=NzQxL...,https,www.bipa.na,/,wpdmact=process&did=NzQxLmhvdGxpbms=,NaN,...,NaN,Namibia,Khomas,Windhoek,17.087500,-22.570560,33763,2022-08-03 12:37:19.336973,geolocation,196.216.38.202
56,56,2021-12-31,JPMorgan Chase & Co. - Luxembourg Stock Exchange,"('Dec 31, 2021',)",http://dl.bourse.lu/dlp/106228d4412e9b4362bbde...,http,dl.bourse.lu,/dlp/106228d4412e9b4362bbde186cac21c232,NaN,NaN,...,LU,Luxembourg,Luxembourg,Luxembourg,6.106240,49.616241,6661,2022-08-03 12:37:19.336989,geolocation,194.154.201.76
57,57,2021-12-31,Banking And Finance Project Topic and material...,"('Jan 1, 2022',)",https://www.easyprojectmaterial.com/category/b...,https,www.easyprojectmaterial.com,/category/banking-and-finance-project-topic-an...,NaN,NaN,...,US,United States,Massachusetts,Burlington,-71.204536,42.505989,46606,2022-08-03 12:37:19.337005,geolocation,192.185.122.45
58,58,2021-12-31,"GLOBE TELECOM, INC. SEC Form 17-A - PDS Group","('Dec 31, 2021',)",https://www.pds.com.ph/wp-content/uploads/2022...,https,www.pds.com.ph,/wp-content/uploads/2022/04/Disclosure-No.-136...,NaN,NaN,...,US,United States,Washington,Seattle,-122.330399,47.601501,19551,2022-08-03 12:37:19.337020,geolocation,107.154.75.6


In [19]:
df_new.head()

,date,header,summary,link,protocol,domain,path,query,fragment,host_name,...,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,IpAddress
0,2022-01-01,December Major Update of ImmuniWeb Discovery,"(Jan 1, 2022,)",https://logon-int.com/blog/december-major-upda...,https,logon-int.com,/blog/december-major-update-of-immuniweb-disco...,,,logon-int.com,...,HK,Hong Kong SAR China,Tsuen Wan,Kwai Chung,114.130257,22.366850,7540,2022-08-03 13:23:19.723632,geolocation,202.181.185.163
1,2022-01-01,I'm not good at New Year's resolution - Davids...,"(Jan 1, 2022,)",https://davidsonsousa.net/blog/post/im-not-goo...,https,davidsonsousa.net,/blog/post/im-not-good-at-new-years-resolution,,,davidsonsousa.net,...,CZ,Czechia,Hlavní město Praha,Prague,14.421610,50.130112,208906,2022-08-03 13:23:19.723686,geolocation,78.24.236.81
2,2022-01-01,ENISA SINGLE PROGRAMMING DOCUMENT 2022–2024,"(Jan 1, 2022,)",https://www.enisa.europa.eu/publications/corpo...,https,www.enisa.europa.eu,/publications/corporate-documents/enisa-single...,,,europa.eu,...,RO,Romania,Bucureşti,Bucharest,26.099350,44.436550,5606,2022-08-03 13:23:19.723705,geolocation,212.146.105.104
3,2022-01-01,Here are 500+ Ivy League courses you can take ...,"(Jan 2, 2022,)",https://www.freecodecamp.org/news/ivy-league-f...,https,www.freecodecamp.org,/news/ivy-league-free-online-courses-a0d7ae675...,,,freecodecamp.org,...,US,United States,Virginia,Ashburn,-77.474197,39.043701,13335,2022-08-03 13:23:19.723722,geolocation,104.26.3.33
4,2022-01-01,Sales - Global Courses,"(Jan 1, 2022,)",https://globalcourses.co.uk/course-category/sa...,https,globalcourses.co.uk,/course-category/sales/,,,globalcourses.co.uk,...,GB,United Kingdom,England,Streatham,-0.114170,51.458328,19527,2022-08-03 13:23:19.723739,geolocation,35.214.88.137
